In [1]:
import os
import os.path as op
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import seaborn as sns
sns.set_theme(style="darkgrid", color_codes=True)
sns.set(font_scale=1.35, style="ticks") #set styling preferences
import statsmodels.api as sm
from scipy import stats
import math
from math import pi
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from scipy.spatial.distance import cdist
from scipy.cluster.vq import kmeans2,vq, whiten
import geopandas as gpd
import h5py
import boto.s3
import glob
import boto3
from zipfile import ZipFile
import shutil
import dask.dataframe as dd
from dask.delayed import delayed
import urllib.request
import gzip
import gcsfs
from urllib.parse import urlparse
from google.cloud import storage
import io

In [2]:
# Read the CSV file into a DataFrame
df_exl = pd.read_excel('C:/Users/nrezaei/Documents/WAV_Scenarios.xlsx')
# Get the paths from the DataFrame
paths = df_exl['Outputs'][0:6].tolist() 

In [3]:
cols_to_use = ['IDMerged', 'tripIndex', 'actStartTime', 'actEndTime','duration_travelling', 'cost_BEAM', 'actStartType', 
               'actEndType', 'duration_walking', 'duration_in_privateCar', 'duration_on_bike', 'duration_in_ridehail', 
              'distance_travelling', 'duration_in_transit', 'distance_walking','distance_bike','distance_ridehail', 
              'distance_privateCar', 'distance_transit', 'mode_choice_planned_BEAM','mode_choice_actual_BEAM',
              'vehicleIds', 'distance_mode_choice', 'replanning_status', 'reason', 'fuel_marginal','BlockGroupStart',
              'startX', 'startY', 'bgid_start', 'tractid_start', 'juris_name_start', 'county_name_start', 'mpo_start', 
               'BlockGroupEnd', 'endX', 'endY', 'bgid_end', 'tractid_end', 'juris_name_end', 'county_name_end', 'mpo_end', 
               'emission_marginal', 'duration_door_to_door', 'waitTime_no_replanning', 'waitTime_replanning', 'actPurpose', 
               'mode_choice_actual_6', 'mode_choice_actual_5', 'mode_choice_actual_4', 'trip_mode_AS_trips', 'logsum_trip_Potential_INEXUS',
               'age', 'income', 'hh_cars', 'TAZ_x', 'origin_x', 'destination_x', 'TAZ_y', 'home_taz', 'auto_ownership', 'home_is_urban', 'home_is_rural', 'DRIVEALONEFREE',
               'DRIVEALONEPAY', 'SHARED2FREE', 'SHARED2PAY', 'SHARED3FREE', 'SHARED3PAY', 'WALK', 'BIKE', 'WALK_LOC', 'WALK_LRF', 
               'WALK_EXP', 'WALK_HVY', 'WALK_COM', 'DRIVE_LOC', 'DRIVE_LRF', 'DRIVE_EXP', 'DRIVE_HVY', 'DRIVE_COM', 'TAXI',
               'TNC_SINGLE', 'TNC_SHARED', 'income_quartiles', 'income_deciles' ] # Specify the columns to read

In [4]:
paths

['gs://beam-core-outputs/wheelchair-feb2023/sfbay-10pop-1fleet/inexus/sfbay_baseline_default-1.0_2021__20230319.csv.gz',
 'gs://beam-core-outputs/wheelchair-feb2023/sfbay-10pop-5fleet/inexus/sfbay_baseline_default-1.0_2021__20230211.csv.gz',
 'gs://beam-core-outputs/wheelchair-feb2023/sfbay-10pop-10fleet/inexus/sfbay_baseline_default-1.0_2021__20230208.csv.gz',
 'gs://beam-core-outputs/wheelchair-feb2023/sfbay-10pop-50fleet/inexus/sfbay_baseline_default-1.0_2021__20230211.csv.gz',
 'gs://beam-core-outputs/wheelchair-feb2023/sfbay-10pop-100fleet/inexus/sfbay_baseline_default-1.0_2021__20230211.csv.gz']

In [5]:
%%time
# Read the CSV files from GCS into pandas DataFrames and add the new columns
dfs = []
for i, path in enumerate(paths):
    df = pd.read_csv(path, compression='gzip')
    df['lever_position_price'] = df_exl.loc[i, 'lever_position_price']
    df['lever_position_fltsz'] = df_exl.loc[i, 'lever_position_fltsz']
    dfs.append(df)

<timed exec>:4: DtypeWarning: Columns (77,131,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,158,167,172,174,175,176,177,179,180,237,238,239,240,249) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:4: DtypeWarning: Columns (77,131,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,158,167,172,174,175,176,177,179,180,237,238,239,240,249) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:4: DtypeWarning: Columns (77,131,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,158,167,172,174,175,176,177,179,180,237,238,239,240,249) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:4: DtypeWarning: Columns (77,131,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,158,167,172,174,175,176,177,179,180,237,238,239,240,249) have mixed types. Specify dtype option on import or set low_me

CPU times: total: 11min 39s
Wall time: 14min 2s


In [8]:
dfs[1].head()

,Unnamed: 0.1,IDMerged,tripIndex,actStartTime,actEndTime,duration_travelling,cost_BEAM,actStartType,actEndType,duration_walking,duration_in_privateCar,duration_on_bike,duration_in_ridehail,distance_travelling,duration_in_transit,distance_walking,distance_bike,distance_ridehail,distance_privateCar,distance_transit,vehicleIds_estimate,mode_choice_planned_BEAM,mode_choice_actual_BEAM,vehicleIds,numPassengers,distance_mode_choice,replanning_status,reason,parkingType,transit_bus,transit_subway,transit_tram,transit_cable_car,ride_hail_pooled,transit_rail,fuelFood,fuelElectricity,fuelBiodiesel,fuelDiesel,fuel_not_Food,fuelGasoline,fuel_marginal,BlockGroupStart,startX,startY,bgid_start,tractid_start,juris_name_start,county_name_start,mpo_start,BlockGroupEnd,endX,endY,bgid_end,tractid_end,juris_name_end,county_name_end,mpo_end,emissionFood,emissionElectricity,emissionDiesel,emissionGasoline,emissionBiodiesel,emission_marginal,duration_door_to_door,waitTime_no_replanning,waitTime_replanning,actPurpose,mode_choice_actual_5,mode_choice_actual_6,mode_choice_actual_4,trip_id,person_id,household_id,tour_id,primary_purpose_x,trip_num,outbound,trip_count,purpose,destination_x,origin_x,destination_logsum_x,depart,trip_mode_AS_trips,logsum_trip_Potential_INEXUS,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination_y,origin_y,household_id_x,tdd,start,end,duration,composition,destination_logsum_y,tour_mode_AS_tours,logsum_tours_mode_AS_tours,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose_y,Unnamed: 0,earning,worker,student,hispanic.1,person_sex,PNUM,sex,household_id_y,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic,DIS,in_wheelchair,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,gt55,seniors,VEHICL,lcm_county_id,hh_children,block_id,gt2,hispanic_head,age_of_head,race_of_head,tenure_mover,hh_size,sf_detached,tenure,hh_cars,income,hh_age_of_head,serialno,num_workers,hh_race_of_head,hh_income,recent_mover,hh_workers,hispanic_status_of_head,hh_seniors,hhsize,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED,income_quartiles,income_deciles,lever_position_price,lever_position_fltsz
0,0,1,577,29606.0,29347.0,259.0,0.000000,othmaint,Home,0.0,259.0,0.0,0.0,6572.270,0.0,0.000,0.0,0.0,6572.270,0.0,

In [7]:
# Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [9]:
def generate_summary_table(df):
    df['year'] = 2018
    df['socialCarbonCost'] = df['emission_marginal']*185
    df['incomeInThousands'] = df['income']/1000
    df = df[df['incomeInThousands'].notna()]
    person_income = pd.pivot_table(df, index=['IDMerged'], aggfunc={'incomeInThousands': lambda x: ', '.join(set(x.dropna().astype(str)))}).reset_index() 
    person_income['incomeInThousands'] = person_income['incomeInThousands'].astype(float)
   
    # Add a column of income ranks
    twenty_one_ranks = person_income['incomeInThousands'].quantile([0, 0.048, 0.095, 0.143, 0.191, 0.239, 0.287, 0.335, 0.383, 0.431, 0.479,
                                                                0.527, 0.575, 0.623, 0.671, 0.719, 0.767, 0.815, 0.863, 0.911, 0.959,
                                                                1]).tolist()
    
    # Add incomeInThousands twenty_one_ranks
    conditions  = [(person_income['incomeInThousands'] >= twenty_one_ranks[0]) & (person_income['incomeInThousands'] < twenty_one_ranks[1]), 
               (person_income['incomeInThousands'] >= twenty_one_ranks[1]) & (person_income['incomeInThousands'] < twenty_one_ranks[2]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[2]) & (person_income['incomeInThousands'] < twenty_one_ranks[3]),
               (person_income['incomeInThousands'] >= twenty_one_ranks[3]) & (person_income['incomeInThousands'] < twenty_one_ranks[4]), 
               (person_income['incomeInThousands'] >=  twenty_one_ranks[4]) & (person_income['incomeInThousands'] < twenty_one_ranks[5]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[5]) & (person_income['incomeInThousands'] < twenty_one_ranks[6]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[6]) & (person_income['incomeInThousands'] < twenty_one_ranks[7]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[7]) & (person_income['incomeInThousands'] < twenty_one_ranks[8]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[8]) & (person_income['incomeInThousands'] < twenty_one_ranks[9]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[9]) & (person_income['incomeInThousands'] <= twenty_one_ranks[10]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[10]) & (person_income['incomeInThousands'] <= twenty_one_ranks[11]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[11]) & (person_income['incomeInThousands'] <= twenty_one_ranks[12]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[12]) & (person_income['incomeInThousands'] <= twenty_one_ranks[13]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[13]) & (person_income['incomeInThousands'] <= twenty_one_ranks[14]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[14]) & (person_income['incomeInThousands'] <= twenty_one_ranks[15]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[15]) & (person_income['incomeInThousands'] <= twenty_one_ranks[16]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[16]) & (person_income['incomeInThousands'] <= twenty_one_ranks[17]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[17]) & (person_income['incomeInThousands'] <= twenty_one_ranks[18]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[18]) & (person_income['incomeInThousands'] <= twenty_one_ranks[19]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[19]) & (person_income['incomeInThousands'] <= twenty_one_ranks[20]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[20]) & (person_income['incomeInThousands'] <= twenty_one_ranks[21])]
    choices = [0, 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    
    person_income['incRank'] = np.select(conditions, choices, default=None)
    
    df = pd.merge(left = df, right = person_income, how='left', on = ['IDMerged'], suffixes=('', '_drop'))
    
    df.drop([col for col in df.columns if 'drop' in col], axis=1, inplace=True)
    
    # Mean pivot table
    mean_inc_table = pd.pivot_table(df,index=['incRank'], aggfunc={'incomeInThousands': np.mean,
           'logsum_trip_Potential_INEXUS': np.mean,
            'duration_travelling': np.mean,
            'duration_door_to_door': np.mean,
            'duration_walking': np.mean,
            'duration_in_privateCar': np.mean,
            'duration_on_bike': np.mean,
            'duration_in_ridehail': np.mean,
            'duration_in_transit': np.mean,
            'waitTime_no_replanning': np.mean,
            'waitTime_replanning': np.mean,
            'distance_travelling': np.mean,
            'distance_walking': np.mean,
            'distance_bike': np.mean,
            'distance_ridehail': np.mean,
            'distance_privateCar': np.mean,
            'distance_transit': np.mean,
            'distance_mode_choice': np.mean,
            'replanning_status': np.mean,
            'fuel_marginal': np.mean,
            'emission_marginal': np.mean,
            'cost_BEAM': np.mean,
            'socialCarbonCost':np.mean,
           'lever_position_price':lambda x: ', '.join(set(x.dropna().astype(str))),
           'lever_position_fltsz':lambda x: ', '.join(set(x.dropna().astype(str))),
           'IDMerged': 'nunique',
           'tripIndex': 'nunique'}).reset_index()
    # Median summary table
    median_inc_table = pd.pivot_table(df, index=['incRank'], aggfunc={'incomeInThousands': np.median,
           'logsum_trip_Potential_INEXUS': np.median,
            'duration_travelling': np.median,
            'duration_door_to_door': np.median,
            'duration_walking':np.median,
            'duration_in_privateCar': np.median,
            'duration_on_bike': np.median,
            'duration_in_ridehail': np.median,
            'duration_in_transit': np.median,
            'waitTime_no_replanning': np.median,
            'waitTime_replanning': np.median,
            'distance_travelling': np.median,
            'distance_walking': np.median,
            'distance_bike': np.median,
            'distance_ridehail': np.median,
            'distance_privateCar': np.median,
            'distance_transit': np.median,
            'distance_mode_choice': np.median,
            'replanning_status': np.median,
            'fuel_marginal': np.median,
            'emission_marginal': np.median,
            'cost_BEAM': np.median,
            'socialCarbonCost':np.median
           }).reset_index() 
   # Sum summary table
    sum_inc_table = pd.pivot_table(df, index=['incRank'], aggfunc={'duration_travelling': np.sum,
            'duration_door_to_door': np.sum,
            'duration_walking': np.sum,
            'duration_in_privateCar': np.sum,
            'duration_on_bike': np.sum,
            'duration_in_ridehail': np.sum,
            'duration_in_transit': np.sum,
            'waitTime_no_replanning': np.sum,
            'waitTime_replanning': np.sum,
            'distance_travelling': np.sum,
            'distance_walking': np.sum,
            'distance_bike': np.sum,
            'distance_ridehail': np.sum,
            'distance_privateCar': np.sum,
            'distance_transit': np.sum,
            'distance_mode_choice': np.sum,
            'replanning_status': np.sum,
            'fuel_marginal': np.sum,
            'emission_marginal': np.sum,
            'cost_BEAM': np.sum,
            'socialCarbonCost':np.sum, 
            'in_wheelchair': lambda x: sum(x == True)                                                        
            }).reset_index() 
            
    mode_counts = df.groupby(['incRank', 'mode_choice_actual_BEAM'])['mode_choice_actual_BEAM'].count().unstack().add_prefix('mode_').reset_index()
    sum_inc_table = sum_inc_table.merge(mode_counts, on='incRank').assign(mode_ridehail_total = lambda x: x['mode_ride_hail'] + x['mode_ride_hail_pooled'])
    sum_inc_table.columns = [col + '_sum' for col in sum_inc_table.columns]
    sum_inc_table = sum_inc_table.rename(columns={'incRank_sum': 'incRank'})
    mm_inc_table = pd.merge(mean_inc_table, median_inc_table, on='incRank', suffixes=('_mean', '_median'))
    summary_inc_table = pd.merge(mm_inc_table, sum_inc_table, on='incRank', how='left')
    summary_inc_table = summary_inc_table.rename(columns={'tripIndex': 'n_trips', 
                                                      'IDMerged': 'n_agents', 
                                                      'logsum_trip_Potential_INEXUS_median': 'Potential_INEXUS_median',
                                                      'logsum_trip_Potential_INEXUS_mean': 'Potential_INEXUS_mean',
                                                      'mode_ride_hail_sum': 'mode_ride_hail_solo_sum'})        
    
    summary_inc_table = summary_inc_table.iloc[:, :2].join(summary_inc_table.iloc[:, 2:].sort_index(axis=1)) 
    
    # shift column 'person' to first position
    third_column = summary_inc_table.pop('n_trips')
    
    # insert column using insert(position,column_name,first_column) function
    summary_inc_table.insert(2, 'n_trips', third_column)
    
    # calculate median and mean wait times by mode and incRank
    grouped_waitTime_no_replanning = df.groupby(['mode_choice_actual_BEAM', 'incRank'])['waitTime_no_replanning'].agg(['mean', 'median']).reset_index()
    
    # pivot the wait time statistics by incRank and mode
    waittime_pivot = pd.pivot_table(grouped_waitTime_no_replanning, index=['incRank'], columns=['mode_choice_actual_BEAM'],
    values=['median', 'mean'], aggfunc=np.sum)
    
    # flatten the multi-index column names and rename them with appropriate suffixes
    waittime_pivot.columns = [f"waitTime_no_replanning_{mode}_{agg}" for (agg, mode) in waittime_pivot.columns.to_flat_index()]
    
    # merge the wait time statistics pivot table with the original pivot table
    summary_inc_table = summary_inc_table.merge(waittime_pivot, on='incRank', how='left')
    
    # calculate median and mean wait times by mode and incRank
    grouped_waitTime_replanning = df.groupby(['mode_choice_planned_BEAM', 'incRank'])['waitTime_replanning'].agg(['mean', 'median']).reset_index()
    
    # pivot the wait time statistics by incRank and mode
    waittime_pivot = pd.pivot_table(grouped_waitTime_replanning, index=['incRank'], columns=['mode_choice_planned_BEAM'], values=['median', 'mean'], aggfunc=np.sum)
    
    # flatten the multi-index column names and rename them with appropriate suffixes
    waittime_pivot.columns = [f"waitTime_replanning_{mode}_{agg}" for (agg, mode) in waittime_pivot.columns.to_flat_index()]
    
    # merge the wait time statistics pivot table with the original pivot table
    summary_inc_table = summary_inc_table.merge(waittime_pivot, on='incRank', how='left')
    return summary_inc_table

In [10]:
%%time
dfs_summary = []
for df in dfs:
    df_summary = generate_summary_table(df)
    dfs_summary.append(df_summary)

CPU times: total: 18min 20s
Wall time: 18min 20s


In [11]:
%%time
sf_stacked = pd.concat(dfs_summary)

CPU times: total: 0 ns
Wall time: 977 µs


In [12]:
sf_stacked.head()

,incRank,n_agents,n_trips,Potential_INEXUS_mean,Potential_INEXUS_median,cost_BEAM_mean,cost_BEAM_median,cost_BEAM_sum,distance_bike_mean,distance_bike_median,distance_bike_sum,distance_mode_choice_mean,distance_mode_choice_median,distance_mode_choice_sum,distance_privateCar_mean,distance_privateCar_median,distance_privateCar_sum,distance_ridehail_mean,distance_ridehail_median,distance_ridehail_sum,distance_transit_mean,distance_transit_median,distance_transit_sum,distance_travelling_mean,distance_travelling_median,distance_travelling_sum,distance_walking_mean,distance_walking_median,distance_walking_sum,duration_door_to_door_mean,duration_door_to_door_median,duration_door_to_door_sum,duration_in_privateCar_mean,duration_in_privateCar_median,duration_in_privateCar_sum,duration_in_ridehail_mean,duration_in_ridehail_median,duration_in_ridehail_sum,duration_in_transit_mean,duration_in_transit_median,duration_in_transit_sum,duration_on_bike_mean,duration_on_bike_median,duration_on_bike_sum,duration_travelling_mean,duration_travelling_median,duration_travelling_sum,duration_walking_mean,duration_walking_median,duration_walking_sum,emission_marginal_mean,emission_marginal_median,emission_marginal_sum,fuel_marginal_mean,fuel_marginal_median,fuel_marginal_sum,in_wheelchair_sum,incomeInThousands_mean,incomeInThousands_median,lever_position_fltsz,lever_position_price,mode_bike_sum,mode_bike_transit_sum,mode_car_hov2_sum,mode_car_hov3_sum,mode_car_sum,mode_drive_transit_sum,mode_hov2_teleportation_sum,mode_hov3_teleportation_sum,mode_ride_hail_pooled_sum,mode_ride_hail_solo_sum,mode_ridehail_total_sum,mode_walk_sum,mode_walk_transit_sum,replanning_status_mean,replanning_status_median,replanning_status_sum,socialCarbonCost_mean,socialCarbonCost_median,socialCarbonCost_sum,waitTime_no_replanning_mean,waitTime_no_replanning_median,waitTime_no_replanning_sum,waitTime_replanning_mean,waitTime_replanning_median,waitTime_replanning_sum,waitTime_no_replanning_bike_mean,waitTime_no_replanning_bike_transit_mean,waitTime_no_replanning_car_mean,waitTime_no_replanning_car_hov2_mean,waitTime_no_replanning_car_hov3_mean,waitTime_no_replanning_drive_transit_mean,waitTime_no_replanning_hov2_teleportation_mean,waitTime_no_replanning_hov3_teleportation_mean,waitTime_no_replanning_ride_hail_mean,waitTime_no_replanning_ride_hail_pooled_mean,waitTime_no_replanning_walk_mean,waitTime_no_replanning_walk_transit_mean,waitTime_no_replanning_bike_median,waitTime_no_replanning_bike_transit_median,waitTime_no_replanning_car_median,waitTime_no_replanning_car_hov2_median,waitTime_no_replanning_car_hov3_median,waitTime_no_replanning_drive_transit_median,waitTime_no_replanning_hov2_teleportation_median,waitTime_no_replanning_hov3_teleportation_median,waitTime_no_replanning_ride_hail_median,waitTime_no_replanning_ride_hail_pooled_median,waitTime_no_replanning_walk_median,waitTime_no_replanning_walk_transit_median,waitTime_replanning_bike_mean,waitTime_replanning_bike_transit_mean,waitTime_replanning_car_mean,waitTime_replanning_car_hov2_mean,waitTime_replanning_car_hov3_mean,waitTime_replanning_drive_transit_mean,waitTime_replanning_hov2_teleportation_mean,waitTime_replanning_hov3_teleportation_mean,waitTime_replanning_ride_hail_mean,waitTime_replanning_ride_hail_pooled_mean,waitTime_replanning_walk_mean,waitTime_replanning_walk_transit_mean,waitTime_replanning_bike_median,waitTime_replanning_bike_transit_median,waitTime_replanning_car_median,waitTime_replanning_car_hov2_median,waitTime_replanning_car_hov3_median,waitTime_replanning_drive_transit_median,waitTime_replanning_hov2_teleportation_median,waitTime_replanning_hov3_teleportation_median,waitTime_replanning_ride_hail_median,waitTime_replanning_ride_hail_pooled_median,waitTime_replanning_walk_median,waitTime_replanning_walk_transit_median
0,0,30543,107367,-0.514907,-0.267716,0.874094,0.356033,93848.875439,70.972145,0.0,7620066.244,7976.246427,5015.053000,8.563857e+08,6619.182992,4046.8480,7.106818e+08,92.064780,0.0,9

In [33]:
sf_stacked.to_csv('C:/Users/nrezaei/Documents/sf_2018_stacked_WAV_incRank.csv', index=False)

In [34]:
sf_stacked.to_csv('s3://beam-core-act/deepDive/CleanData/SanFrancisco/Stacked/sf_2018_stacked_WAV_incRank.csv', index=False)

NoCredentialsError: Unable to locate credentials

In [21]:
%%time
stacked_rh_path = "https://beam-core-act.s3.amazonaws.com/deepDive/CleanData/SanFrancisco/Stacked/"
sf_stacked_12 = pd.read_csv(stacked_rh_path + 'sf_2018_stacked_rh_fltsz_price_incRank_1_2.csv')

CPU times: total: 15.6 ms
Wall time: 77.1 ms


In [24]:
frames = [sf_stacked_12, sf_stacked]
sf_stacked123 = pd.concat(frames)

In [25]:
sf_stacked123.to_csv('s3://beam-core-act/deepDive/CleanData/SanFrancisco/Stacked/sf_2018_stacked_rh_fltsz_price_incRank_1_2_3.csv', index=False)